In [1]:
import torch
import os
import numpy as np

In [2]:
one_train_dir = os.path.join('dataset', 'part_one_dataset', 'train_data')
one_eval_dir = os.path.join('dataset', 'part_one_dataset', 'eval_data')
two_train_dir = os.path.join('dataset', 'part_two_dataset', 'train_data')
two_eval_dir = os.path.join('dataset', 'part_two_dataset', 'eval_data')

In [3]:
# domains = [{} for _ in range(20)]
# eval_domains = [{} for _ in range(20)]

# for i in range(10):
#     domains[i] = torch.load(os.path.join(one_train_dir, f'{i+1}_train_data.tar.pth'), weights_only=False)
#     domains[i+10] = torch.load(os.path.join(two_train_dir, f'{i+1}_train_data.tar.pth'), weights_only=False)
#     eval_domains[i] = torch.load(os.path.join(one_eval_dir, f'{i+1}_eval_data.tar.pth'), weights_only=False)
#     eval_domains[i+10] = torch.load(os.path.join(two_eval_dir, f'{i+1}_eval_data.tar.pth'), weights_only=False)
    
#     domains[i]['data'] = domains[i]['data'].reshape(2500, -1)
#     domains[i+10]['data'] = domains[i+10]['data'].reshape(2500, -1)
#     eval_domains[i]['data'] = eval_domains[i]['data'].reshape(2500, -1)
#     eval_domains[i+10]['data'] = eval_domains[i+10]['data'].reshape(2500, -1)

In [4]:
train_paths = [os.path.join(one_train_dir, f'{i+1}_train_data.tar.pth') for i in range(10)] + [os.path.join(two_train_dir, f'{i+1}_train_data.tar.pth') for i in range(10)]
eval_paths = [os.path.join(one_eval_dir, f'{i+1}_eval_data.tar.pth') for i in range(10)] + [os.path.join(two_eval_dir, f'{i+1}_eval_data.tar.pth') for i in range(10)]

eval_domains = [torch.load(eval_paths[i], weights_only=False) for i in range(20)]

## Model

In [5]:
from lwp import LWP

## Training and Predictions

In [6]:
from sklearn.metrics import accuracy_score
import pandas as pd

model = LWP(distance_metric='cosine')

df = pd.DataFrame()

for idx,train_path in enumerate(train_paths):
    
    domain = torch.load(train_path, weights_only=False)    
    x_test = domain['data'].reshape(2500, -1)
    y_pred = model.predict(x_test) if 'targets' not in domain else domain['targets']
    
    model.fit(x_test, y_pred)
    print(model.class_counts)
    del domain
    
    scores = []
    for eval_domain in eval_domains[:idx+1]:
        
        features = eval_domain['data'].reshape(2500, -1)
        labels = eval_domain['targets']
        
        preds = model.predict(features)
        acc = accuracy_score(labels, preds)
        
        scores.append(acc)
        
    df[f'Domain {idx+1}'] = scores + [np.nan] * (20 - len(scores))

{0: 253, 1: 243, 2: 255, 3: 244, 4: 262, 5: 236, 6: 250, 7: 253, 8: 254, 9: 250}
{0: 454, 1: 385, 2: 421, 3: 309, 4: 496, 5: 757, 6: 458, 7: 466, 8: 544, 9: 710}
{0: 678, 1: 542, 2: 584, 3: 403, 4: 804, 5: 1198, 6: 682, 7: 701, 8: 845, 9: 1063}
{0: 871, 1: 703, 2: 736, 3: 537, 4: 1091, 5: 1613, 6: 906, 7: 962, 8: 1113, 9: 1468}
{0: 1064, 1: 864, 2: 917, 3: 700, 4: 1372, 5: 1983, 6: 1141, 7: 1229, 8: 1375, 9: 1855}
{0: 1253, 1: 1030, 2: 1121, 3: 883, 4: 1640, 5: 2373, 6: 1368, 7: 1515, 8: 1644, 9: 2173}
{0: 1444, 1: 1205, 2: 1309, 3: 1070, 4: 1907, 5: 2758, 6: 1602, 7: 1779, 8: 1907, 9: 2519}
{0: 1617, 1: 1381, 2: 1511, 3: 1268, 4: 2148, 5: 3126, 6: 1843, 7: 2051, 8: 2167, 9: 2888}
{0: 1794, 1: 1541, 2: 1712, 3: 1458, 4: 2433, 5: 3460, 6: 2090, 7: 2299, 8: 2445, 9: 3268}
{0: 1992, 1: 1717, 2: 1927, 3: 1674, 4: 2723, 5: 3791, 6: 2305, 7: 2558, 8: 2717, 9: 3596}
{0: 2176, 1: 1890, 2: 2185, 3: 1870, 4: 2962, 5: 4124, 6: 2541, 7: 2821, 8: 2990, 9: 3941}
{0: 2357, 1: 2063, 2: 2428, 3: 2071, 

## Evaluation

In [7]:
df

,Domain 1,Domain 2,Domain 3,Domain 4,Domain 5,Domain 6,Domain 7,Domain 8,Domain 9,Domain 10,Domain 11,Domain 12,Domain 13,Domain 14,Domain 15,Domain 16,Domain 17,Domain 18,Domain 19,Domain 20
0,0.2916,0.2776,0.2736,0.2724,0.2700,0.2692,0.2664,0.2624,0.2608,0.2600,0.2604,0.2604,0.2592,0.2576,0.2568,0.2568,0.2564,0.2564,0.2560,0.2576
1,NaN,0.2720,0.2776,0.2704,0.2684,0.2680,0.2664,0.2644,0.2640,0.2628,0.2608,0.2588,0.2572,0.2560,0.2560,0.2552,0.2536,0.2540,0.2528,0.2524
2,NaN,NaN,0.2716,0.2656,0.2672,0.2644,0.2632,0.2624,0.2620,0.2608,0.2596,0.2588,0.2592,0.2592,0.2588,0.2568,0.2588,0.2544,0.2540,0.2552
3,NaN,NaN,NaN,0.2748,0.2700,0.2700,0.2672,0.2676,0.2672,0.2648,0.2648,0.2656,0.2648,0.2648,0.2644,0.2644,0.2640,0.2616,0.2620,0.2584
4,NaN,NaN,NaN,NaN,0.2964,0.2928,0.2912,0.2904,0.2904,0.2900,0.2868,0.2868,0.2856,0.2840,0.2836,0.2832,0.2832,0.2824,0.2820,0.2816
5,NaN,NaN,NaN,NaN,NaN,0.2640,0.2644,0.2624,0.2616,0.2616,0.2616,0.2620,0.2616,0.2612,0.2620,0.2612,0.2608,0.2584,0.2568,0.2572
6,NaN,NaN,NaN,NaN,NaN,NaN,0.2612,0.2644,0.2676,0.2676,0.2656,0.2644,0.2640,0.2648,0.2628,0.2624,0.2632,0.2612,0.2620,0.2600
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2700,0.2728,0.2732,0.2728,0.2736,0.2712,0.2704,0.2696,0.2680,0.2680,0.2668,0.2680,0.2668
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2608,0.2616,0.2592,0.2592,0.2564,0.2568,0.2556,0.2556,0.2556,0.2548,0.2548,0.2516
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2612,0.2596,0.2584,0.2564,0.2564,0.2564,0.2564,0.2552,0.2560,0.2540,0.2520
